In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import itertools

from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

g:\Code\Python\GitHub\maboo\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_4 = pd.read_csv('G:/Code/Python/GitHub/maboo/Brand/brand_data/brand_data_4.csv')
data_1 = pd.read_csv('G:/Code/Python/GitHub/maboo/Brand/brand_data/brand_data_1.csv')
data_1 = data_1.iloc[:, :2]
valid_brand = [i for i in set(data_1['brand'].to_list())]

data = pd.concat([data_1, data_4])
data.reset_index(drop=True, inplace=True)

valid_index = []
for i in range(len(data)):
    if data['brand'][i] in valid_brand:
        data['brand'][i] = 1
        valid_index.append(i)
    else:
        data['brand'][i] = 0

# X = data['name']
# y = le.fit_transform(data['bra_整合'])
# X_train, X_val, y_train, y_val = train_test_split(data['name'].values, data['brand'].values, test_size=0.2, random_state=89)

In [3]:
X_train = data[data['brand'] == 1]['name']
y_train = data[data['brand'] == 1]['brand']

X_valid = data[data['brand'] == 0]['name']
y_valid = data[data['brand'] == 0]['brand']

In [4]:
from data_preprocess import BrandDataset
from cvdd_net import CVDDNet
from training_pipe import CVDDTrainer

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
train_dataset = BrandDataset(X_train, y_train, tokenizer, 512)
valid_dataset = BrandDataset(X_valid, y_valid, tokenizer, 512)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32)

In [6]:
net = CVDDNet('bert-base-multilingual-uncased', 150, 3)
trainer = CVDDTrainer('adam', 0.001, 150, lambda_p=1.0, alpha_scheduler='logarithmic',
                      weight_decay=0.5e-6, device='cuda')

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
trainer.train(train_loader, net)

g:\Code\Python\GitHub\maboo\.venv\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


KeyboardInterrupt: 

In [20]:
indices = [1, 2, 3]
for idx in indices:
    tokens = train_loader.dataset[idx]['ids']
    text = 
    

tensor([1453,  575, 2652, 3835, 1467,   95, 7555, 6388, 2073, 7347, 4355,  297,
        1007, 7970, 8027, 7907, 5573, 4707, 5101, 4706, 1511, 5539, 7618, 6707,
        7341, 5234, 2786, 4771, 6986, 1927, 7313, 1538])


In [52]:
tokens = train_loader.dataset[0]['ids']

In [53]:
tokenizer.decode(tokens, True)

'abs 深 層 極 淨 保 濕 竹 炭 卸 妝 油'